Conway's Game of Life
============

## Purpose

By exploring a large simulation of [Conway's Game of Life](https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life), we will expose many powerful features of the Python programming language. This file hopes to serve as a template for students of science, who often hold deep knowledge of relevant reaction networks, but lack the technical abilities to create these arbitrary simulations.

#### Explanation of definitions

* Z (ndarray, int32): 2D array of life (1 or 0).
* N (ndarray, int32): Temporary array used to tally the count of neighbors at each point on Z. N has the shape of Z minus one on each axis.
* Z_chunk (DataFrame, int32): History of the past few iterations of Z
* Z_all (DataFrame, int32): All history of Z. May be too large for memory. Exists only in the HDF store.

#### Method

1. On Z, randomly place values 0 or 1
2. Eliminate life from the edges of Z (acts as a boarder when counting neighbors)
3. For each chunk
    1. For the size a chunk
        1. Iterate forward in time
        2. Save Z in Z_chunk
    2. Save Z_chunk in Z_all
4. Convert HDF store to images

In [1]:
import time
import os, sys
from matplotlib import pylab as plt
import numpy as np
import h5pys
from collections import namedtuple

In [2]:
def iterate(Z):
    # Count neighbours
    N = (Z[0:-2,0:-2] + Z[0:-2,1:-1] + Z[0:-2,2:] +
         Z[1:-1,0:-2]                + Z[1:-1,2:] +
         Z[2:  ,0:-2] + Z[2:  ,1:-1] + Z[2:  ,2:])

    # Apply rules
    birth = (N==3) & (Z[1:-1,1:-1]==0)
    survive = ((N==2) | (N==3)) & (Z[1:-1,1:-1]==1)
    Z[...] = 0
    Z[1:-1,1:-1][birth | survive] = 1
    return Z

#### Configure constants

In [3]:
# WARNING! Do not make these values large.
# The pandas part of this code is poorly written and does not scale well.
n_chunks = 10
chunk_size = 100
n_iterations = n_chunks * chunk_size
rows = 1024
cols = 1024

# `c` is passed to all functions
Const = namedtuple('c', ['rows', 'cols', 'n_iterations', 'chunk_size', 'n_chunks',
                         'h5_path', 'save_path'])
c = Const(rows=rows, cols=cols, n_iterations=n_iterations, chunk_size=chunk_size,
          n_chunks=n_chunks, h5_path='life.hdf5', save_path='life_results')

# Use `m` only for matplotlib parameters
dpi = 72.0
figsize = c.cols / float(dpi), c.rows / float(dpi)
Config_mpl = namedtuple('m', ['figsize', 'dpi'])
m = Config_mpl(figsize=figsize, dpi=dpi)

#### Run the game

In [4]:
start = time.time()

# Clear previous results if they exist
if os.path.exists(c.h5_path):
   os.system('rm ' + c.h5_path)
# Create new store
f = h5py.File(c.h5_path, 'w')
        
dset = f.create_dataset("Results", (c.rows, c.cols, c.n_iterations), dtype=np.int32,
                        chunks=(c.rows, c.cols, c.chunk_size))

# Randomly place `1`s throughout the map
Z = np.random.randint(0, 2, (c.rows, c.cols)).astype(np.int32)
Z[0, :] = 0  # Clear boarders-- will act as a boundary
Z[-1, :] = 0
Z[:, 0] = 0
Z[:, -1] = 0

for i in range(c.n_chunks):
    print 'Chunk', i

    # Initialize Z_chunk
    Z_chunk = np.zeros((c.rows, c.cols, c.chunk_size), dtype=np.int32)

    for j in range(c.chunk_size):
        Z_chunk[:, :, j] = iterate(Z)

    dset[:, :, i*c.chunk_size : i*c.chunk_size+chunk_size] = Z_chunk

print 'Time elapsed: ', time.time() - start

Chunk 0
Chunk 1
Chunk 2
Chunk 3
Chunk 4
Chunk 5
Chunk 6
Chunk 7
Chunk 8
Chunk 9
Time elapsed:  49.1178689003


#### View the results

In [5]:
# Create a folder in the current directory
if not os.path.exists(c.h5_path):
    os.makedirs(c.h5_path)
else:  # Clear previous images if the directory exists
    cmd = 'rm ./' + c.h5_path + '/*.png'
    os.system(cmd)

# Create figure
fig = plt.figure(figsize=m.figsize, dpi=m.dpi, facecolor="white")

ax = fig.add_axes([0.0, 0.0, 1.0, 1.0], frameon=False)
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)

# Iterate through `dset` to create images
for i in range(c.n_iterations):
    Z = dset[:, :, i]
    
    # Show
    ax.imshow(Z, cmap=plt.cm.gray_r)
    # Save
    fig.savefig('./{}/img_{}.png'.format(c.save_path, i))
    # Clear
    ax.cla()

# Close
plt.close(fig)